In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

dataset_loc = '../routerbench/dataset/routerbench_0shot.csv'

In [17]:
def process_dataset(model):
    routerbench_0shot = pd.read_csv(dataset_loc)

    # Create a special dataframe with the specified columns
    model_df = routerbench_0shot[['sample_id', 'prompt', 'eval_name'] + [col for col in routerbench_0shot.columns if model in col]]
    
    # We remove Chinese samples because we are using English-only fasttext
    model_df = model_df[~model_df['eval_name'].str.contains('chinese', case=False, na=False)]
    
    # print(model_df[model].value_counts())
    # Since we use a classifier, we remove the few (49 for gpt-3.5-turbo-1106) samples with a score not in {0, 0.25, 0.5, 0.75, 1}
    model_df = model_df[model_df[model].isin([0, 0.25, 0.5, 0.75, 1])]
    train_df, test_df = train_test_split(model_df[['prompt', model]], test_size=0.3, random_state=42)
    
    os.makedirs(f'{model.replace("/", "-")}', exist_ok=True)
    train_df.to_csv(f'{model.replace("/", "-")}/train.csv', index=False, header=["prompt", "score"])
    test_df.to_csv(f'{model.replace("/", "-")}/test.csv', index=False, header=["prompt", "score"])



In [18]:
header_list = pd.read_csv(dataset_loc, nrows=0).columns.tolist()
header_list
for header in header_list:
    if not any(exclude in header for exclude in ['sample_id', 'prompt', 'eval_name', 'oracle_model_to_route_to', 'cost', 'model_response']):
        print(header)
        process_dataset(header)

WizardLM/WizardLM-13B-V1.2
claude-instant-v1
claude-v1
claude-v2
gpt-3.5-turbo-1106
gpt-4-1106-preview
meta/code-llama-instruct-34b-chat
meta/llama-2-70b-chat
mistralai/mistral-7b-chat
mistralai/mixtral-8x7b-chat
zero-one-ai/Yi-34B-Chat
